## 5.5 계층별 고객 판매 전략 (클러스터링, 차원 축소)


### 공통 전처리

In [ ]:
# 공통 처리

# 불필요한 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 한글 글꼴 설정
import platform

if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':
    plt.rc('font', family='Apple Gothic')

# 데이터프레임 출력용 함수
from IPython.display import display

# 숫자 출력 조정
# 넘파이 부동소수점 출력 자리수 설정
np.set_printoptions(suppress=True, precision=4)

# 판다스 부동소수점 출력 자리수 설정
pd.options.display.float_format = '{:.4f}'.format

# 데이터프레임 모든 필드 출력
pd.set_option("display.max_columns",None)

# 그래프 글꼴 크기 설정
plt.rcParams["font.size"] = 14

# 난수 시드
random_seed = 123

데이터 집합 배포 웹 페이지  

https://archive.ics.uci.edu/ml/datasets/wholesale+customers

### 5.5.4 데이터 읽어 들이기부터 데이터 확인까지

#### 데이터 읽어 들이기

In [ ]:
# 데이터 읽어 들이기
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00292/Wholesale%20customers%20data.csv'

df = pd.read_csv(url)

# 우리말 필드명 정의
columns = [ '판매_채널', '지역', '신선식품', '유제품', '식료품',
    '냉동식품', '세제_종이제품', '부식']
df.columns = columns

#### 데이터 확인하기

In [ ]:
# 데이터 내용 확인
display(df.head())

# 건수 확인
print(df.shape)

In [ ]:
# 누락 값 확인
print(df.isnull().sum())

In [ ]:
# '판매_채널'의 레이블값 확인
print(df['판매_채널'].value_counts())
print()

# '지역'의 레이블값 확인
print(df['지역'].value_counts())

#### '판매_채널' 레이블 값의 의미
1.  Horeca(호텔, 레스토랑, 카페) 298 
2. Retail(소매점) 142 

#### '지역' 레이블 값의 의미
1.  Lisbon(리스본) 77 
2. Oporto(포르투) 47 
3. Other Region(기타) 316 



#### 히스토그램 그리기

In [ ]:
# '판매_채널'과 '지역'을 제외
df2 = df.drop(['판매_채널', '지역'], axis=1)

# 대상 필드의 히스토그램을 그리기
from pylab import rcParams
rcParams['figure.figsize'] = (8, 8)
df2.hist(bins=20)
plt.tight_layout() 
plt.show()

### 5.5.5 클러스터링

In [ ]:
# 데이터 전처리 및 데이터 분할은 불필요함

# 알고리즘 선택
from sklearn.cluster import KMeans

# 클러스터 수를 지정
clusters=4

# 알고리즘 정의
algorithm = KMeans(n_clusters=clusters, 
    random_state=random_seed)

In [ ]:
# 학습 및 예측
y_pred = algorithm.fit_predict(df2)

# 결과 중 일부를 확인
print(y_pred[:20])

### 5.5.6 클러스터링 결과 분석

#### 그룹별 평균 계산하기

In [ ]:
# 그룹별 평균 계산하기
df_cluster = df2.groupby(y_pred).mean()
display(df_cluster)

#### 그룹별 평균 그래프 그리기

In [ ]:
# 그룹별 평균을 막대그래프로
df_cluster.plot(kind='bar',stacked=True,
    figsize=(10, 6),colormap='jet')
plt.show()

#### 분석 결과

* **0**: 신선식품 중심
* **1**: 식료품 중심
* **2**: 매량구매
* **3**: 소량구매

#### 그룹과 판매 채널, 지역의 관계

In [ ]:
# 판매_채널, 지역 필드와 클러스터의 관계

# 판매_채널, 지역 필드를 df3으로 추출
df3 = df[['판매_채널', '지역']]

# 그래프 크기를 설정
rcParams['figure.figsize'] = (6,3)

# 그룹별로 그래프를 그림
for i in range(clusters):
    fig, ax = plt.subplots()
    w = df3[y_pred==i]
    print(f'====== 그룹{i} ======')
    w.hist(ax=ax)
    plt.tight_layout()
    plt.show()

#### 분석 결과
그룹0(신선 그룹)과 그룹3(소량구매 그룹)은 판매 채널 1(호텔/레스토랑/카페)과 관련이 깊다.

그룹1(식품 그룹)과 그룹2(대량구매 그룹)은 판매 채널 2(소매점)와 관련이 깊다.

지역과 그룹은 특별히 관련이 없어 보인다.

### 5.5.7 차원축소

In [ ]:
# 알고리즘 선택
from sklearn.decomposition import PCA

# 모델 생성
# 산점도를 그리는 것이 목적이므로 2차원으로 축소
pca = PCA(n_components=2)

In [ ]:
# 학습 및 변환
d2 = pca.fit_transform(df2)

# 결과 중 일부를 출력
print(d2[:5,:])

### 5.5.8 차원축소를 활용하는 방법

#### 산점도 그리기

In [ ]:
# 그룹별로 색을 달리해 산포도 그리기

plt.figure(figsize=(8,8))
marks = ['.', 'x', '*', '+']
labels = ['신선', '식품', '대량', '소량']
colors = ['grey', 'lightblue', 'blue', 'black']
for i in range(clusters):
  plt.scatter(d2[y_pred==i][:,0], d2[y_pred==i][:,1], 
    marker=marks[i], label=labels[i], s=100, c=colors[i])
plt.legend(fontsize=14)
plt.show()

#### 이상값 조사하기

In [ ]:
# 신선 그룹의 이상값 조사하기
display(df[d2[:,0] > 100000])

**고찰**

인덱스 181번 고객은 신선식품의 구매량이 특히 많다고 볼 수 있다.

In [ ]:
# 대량구매 그룹의 이상값 조사하기
display(df[d2[:,1] > 80000])

**고찰**
인덱스 85번 고객은 특히 식료품, 세제_종이제품의 구매액이 매우 높다.

In [ ]:
# 통계정보를 확인
display(df2.describe())